# The grid as an object

In this notebook, the grid object and the xarray dataset and data frame are explained and demonstrated with a few illustrating examples. 

---

The module grid.py contains one class, `Grid()`. When we import the module, a few convenient variables are also generated. Most packages are imported with the module, but some rarely used dependencies are imported in functions when needed. More about this later. 

Dependencies: 

`basemap geopandas matplotlib numpy pyproj rasterio scipy xarray`

and for some functions:

`bokeh fiona holoviews imageio mayavi shapely `

We import grid: 

In [1]:
#Depending on where the grid.py is located, you cane either call it from its location...

config_file = "../grid_code/agrid.py"
with open(config_file) as f:
    code = compile(f.read(), config_file, 'exec')
    exec(code, globals(), locals())

# ... or use you Python path or present working directory:
#from agrid import *

# ... or add the package to your python path

print(Grid)
print(km) #-This is so handy! -You're welcome. 

#Check if some modules have been imported
print('numpy' in sys.modules, 
      'rasterio' in sys.modules, 
      'bokeh' in sys.modules, sep='\n') #(bokeh is imported in a function)

print(type(np), type(plt), type(xr)) # Use of standard aliases 

/Users/tobiasstal/anaconda3/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/tobiasstal/anaconda3/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


<class '__main__.Grid'>
1000
True
True
False
<class 'module'> <class 'module'> <class 'module'>


We can now define a grid object. 

Each grid is a model frame with a defended, regular$^1$ spatial extent. The frame is generated when the object is initiated and resolution, coordinate system and coordinates are defined. 

The class defines an object. Let's say that we would like to develop a gridded model of the African continent. We want to use WGS 1984, EPSG:4326, and a resolution of $0.5^\circ$. We define the extent as left, right, up, down in the units of the projection, degrees in this example. 

When initiating the object, a number of instance variables are defined. These contains the size of the grid, the name of the grid and the affine transform of the grid. (`use_dask` will be explained later). 

---
1. This will updated in later versions. 

In [2]:
africa = Grid(crs=4326, left=-20, down= -45, right=55, up=40, 
              res = (0.5, 0.5), use_dask=False)

print(africa)

With the object, variables are generated, both class variables and instance variables. 

E.g the class variable verbose is a switch for print statements from the functions. Instance variables `nx`, `ny`, `nn`, `shape3` are integers and tuples that conveniently gives to fundamental size of the grid. `transform` is the affine transform. 

In [3]:
# Class variables
print(Grid.verbose) # Switch for print commands

# Instance variables
print(africa.ny, africa.nx, africa.nn, africa.shape3)
print(africa.transform)

False
169 149 (169, 149) (169, 149, 5)
| 0.50, 0.00,-19.75|
| 0.00,-0.50, 39.75|
| 0.00, 0.00, 1.00|


Note that dimensions are given as y, x, not x, y. This was a tricky decision to take. The reasoning is, that numpy arrays are given as rows-columns. By given the y dimension first, we are consistent through the project, but I'd expect that this might cause confusion. I'm happy for suggestions of how to simplify the indexing. Indexing is, however, less a problem when we move to the main feature of the grid object, the xarray dataset. 

---

Most important, and xarray dataset is created and it is already populated with dimensions that contains coordinates. X, Y and Z are (in this example) the three spatial dimensions. X and Y are set to the selected projection (epsg:4326) and is hence in degrees. 

Xarray dataset can be a bit tricky to grasp. There is good information at [xarray project page](http://xarray.pydata.org/en/stable/api.html) where there are a number of code examples. As always, [Stack overflow](https://stackoverflow.com/search?q=xarray) is a good resource as well. Think about it as Pandas, but multidimensional. It can also be understood as a very organized way to arrange numpy arrays in relation to each other. Another approach, if you are familiar with [NetCDF](https://www.unidata.ucar.edu/software/netcdf/), xarray has a similar internal structure. 

For our use, each grid object contains one dataset e.g. `africa.ds`. The dataset contains a number of data frames that are linked to coordinates in different dimensions. The dataset also contain attribute data. 

In [ ]:
print(africa.ds)

We use standard Python / numpy methods to generate or process data and assign to the dataset. Here we generate a 2D array of data points and assign it to the Y and X coordinates of the grid. 

In [ ]:
random_data = np.random.random(africa.nn)
africa.ds['RANDOM'] = (('Y', 'X'), random_data)

The object contains a number of functions to import, process, export and visualize data in the grid. More about import functions in next tutorial. For now, we download a global polygon shape file. 

In [ ]:
# Usning standard *nix: 

! mkdir -p ../../data/vector
! wget -nc http://data.openstreetmapdata.com/simplified-land-polygons-complete-3857.zip \
    -O ../../data/vector/simplified-land-polygons-complete-3857.zip
! unzip -n ../../data/vector/simplified-land-polygons-complete-3857.zip -d ../../data/vector

The shape file can be used to generate a Boolean raster. The function `assign_shape` in the object ´africa´ reads a shape file and rasterises it. Here, all polygons are assigned `True` as there is an FID attribute. This might take some time, as the vector layer is very large. This will also generate some warnings, as no vector file is perfect... 

Note again, the order of dimension: Y, X. 

In [ ]:
africa.ds['LAND'] = (('Y', 'X'), 
                     -1 < africa.assign_shape('../../data/vector/simplified-land-polygons-complete-3857/simplified_land_polygons.shp', 'FID') )

Now, if we look at our dataset, we have two data variables, or frames. RANDOM and LAND. Both have an extent along the X and Y axes. 

In [ ]:
africa.ds

Xarray works similar to numpy, in many ways. Here are some examples of arithmetic and conditional computations. We will look closer at the `map_grid` function in later tutorial. 

In [ ]:
africa.ds['BIG_RANDOM'] = africa.ds['RANDOM'] * 2 

africa.ds['DATA'] = africa.ds['BIG_RANDOM']*africa.ds['LAND']

africa.ds['DATA'] = africa.ds['DATA'].where(africa.ds['DATA'] != 0.)  

africa.map_grid('DATA', cmap='viridis') # We only need to send the label to the function 

Xarray have a number option to index and select data. See details about the API at the [Xarray project page](http://xarray.pydata.org/en/stable/api.html). Here is a bunch of examples: 

In [ ]:
# Values can be assigned
africa.ds['DATA'][dict(X=slice(4, 30), Y=slice(40, 60))] = 2

# Unfortunately, sel cannot be used to directely assign values due to a limitation of Python's syntax. 
# https://stackoverflow.com/a/40030606/4535020
# There should be a detour via a dict, but I dinn't get it to work yet:
#africa.ds['DATA'].loc[dict(X=20, Y=-20)] = 1

#modify one grid point, using where() or loc()
africa.ds['DATA'] = xr.where((africa.ds.coords['lat']==-20)&(africa.ds.coords['lon']==30), 2, africa.ds['DATA'])

# Assign value to an area with a mask 
mask = (africa.ds.coords['lat']>-20)&(africa.ds.coords['lat']<-12)&(africa.ds.coords['lon']>16)&(africa.ds.coords['lon']<25)
africa.ds['DATA'] = xr.where(mask, 3, africa.ds['DATA'])

# Data by numpy index:
A =  africa.ds['DATA'][66, 33]

# Coord by index:
B = africa.ds.coords['lon'][66, 33]

# Data by coord values:
C = africa.ds['DATA'].isel(X=33, Y=66)

# Closest to coordinate values:
D = africa.ds['DATA'].sel(X=[14.5, 71.32], method='nearest') 

# Or another coordinate set:
E = africa.ds.coords['lon'][66, 33]

# Use coordinates to select a slice from a data array:
F = africa.ds['DATA'][dict(X=slice(8, 18), Y=slice(58, 68))]

# Select 2D array from slices of coordinates from dict of indicies
G = africa.ds['DATA'][dict(X=slice(120, 150), Y=slice(40, 70))]

# Select 2D array from coordinate values
H = africa.ds['DATA'].sel(X=slice(5, 30), Y=slice(-10, -35))

I = africa.ds['DATA'][10:150,10:150]

fig, axes = plt.subplots(2,2, figsize=(10,10))

for ax, array in zip(axes.flatten(), [F, G, H, I]):
    ax.imshow(array)


plt.show()

This returned a cell of the entire dataset, but we can also extract numpy arrays and numbers by using a dictionary interface `values`: 

In [ ]:
print(type(africa))
print(type(africa.ds))
print(type(africa.ds['DATA']))
print(type(africa.ds['DATA'].values))

Meta data are important:

In [ ]:
# Add attribute data directely
africa.ds.attrs['units'] = 'degrees'
africa.ds.attrs['contact'] = 'mail@address.gh'

#Or as text, xml, json etc
import json
with open('attr.json', 'r') as fp:
    meta_data = json.loads(fp.read())

for item in meta_data.items():
    africa.ds.attrs[item[0]] = item[1]

Handy attributes to xarray dataset: 

In [ ]:
print('Coords:')
print(africa.ds.coords)

print('Attributes, metadata:')
print(africa.ds.attrs)

print('Size in bytes:')
print(africa.ds.nbytes)

We use a function from the class to save. Returns size.  

In [ ]:
africa.save(file_name='africa.nc')

In [ ]:
!stat africa.nc  

Note that the module have very limited error handling, exceptions are raised by Python and the used models. 

In [ ]:
africa.ds['NO_DATA_HERE']

# Raises: "if var_name is None: KeyError: 'NO_DATA'"

Clear memory: 

In [ ]:
#del africa 

In [ ]:
! which Python3

In [65]:
import json



#def toJSON(s):
#        return json.dumps(s, default=lambda o: o.__dict__, 
#            sort_keys=True, indent=4)
    
write_coords = False
    
info = africa.__dict__.copy()
info['ds'] = 'xarray dataset'
info['coord_d_type'] = str(info['coord_d_type'])
for array in ['xv', 'yv', 'lon', 'lat']:
    if write_coords:
        info[array] = info[array].tolist()
    else:
        info[array] = info[array][[0,0,-1,-1],[0,-1,0,-1]].tolist()


with open('data.json', 'w') as outfile:
    json.dump(info, outfile, indent=4, ensure_ascii=False)
        

In [55]:
j

'{\n    "res": [\n        0.5,\n        0.5\n    ],\n    "depths": [\n        0.0,\n        8000.0,\n        16000.0,\n        40000.0,\n        350000.0\n    ],\n    "x1y1x2y2": [\n        -19.75,\n        39.75,\n        54.75,\n        -44.75\n    ],\n    "wsen": [\n        -19.75,\n        -44.75,\n        54.75,\n        39.75\n    ],\n    "left": -19.75,\n    "up": 39.75,\n    "right": 54.75,\n    "down": -44.75,\n    "ul": [\n        -19.75,\n        39.75\n    ],\n    "ur": [\n        54.75,\n        39.75\n    ],\n    "lr": [\n        54.75,\n        -44.75\n    ],\n    "ll": [\n        -19.75,\n        -44.75\n    ],\n    "nx": 149,\n    "ny": 169,\n    "nn": [\n        169,\n        149\n    ],\n    "transform": [\n        0.5,\n        0.0,\n        -19.75,\n        0.0,\n        -0.5,\n        39.75,\n        0.0,\n        0.0,\n        1.0\n    ],\n    "coord_d_type": "<class \'numpy.float32\'>",\n    "ds": "xarray dataset",\n    "nz": 5,\n    "shape2": [\n        169,\n 

In [48]:
j

'{"res":[0.5,0.5],"depths":[0.0,8000.0,16000.0,40000.0,350000.0],"x1y1x2y2":[-19.75,39.75,54.75,-44.75],"wsen":[-19.75,-44.75,54.75,39.75],"left":-19.75,"up":39.75,"right":54.75,"down":-44.75,"ul":[-19.75,39.75],"ur":[54.75,39.75],"lr":[54.75,-44.75],"ll":[-19.75,-44.75],"nx":149,"ny":169,"nn":[169,149],"transform":[0.5,0.0,-19.75,0.0,-0.5,39.75,0.0,0.0,1.0],"coord_d_type":"<class \'numpy.float32\'>","ds":"xarray dataset","nz":5,"shape2":[169,149],"shape3":[169,149,5],"xv":[-19.75,54.75,-19.75,54.75],"yv":[39.75,39.75,-44.75,-44.75],"crs_src":4326,"crs":4326,"lon":[-19.75,54.75,-19.75,54.75],"lat":[39.75,39.75,-44.75,-44.75]}'